In [1]:
qi("AmpelContext", "AmpelDB", "AlertProcessor", "AmpelLogger", "TarAlertLoader")

In [2]:
ctx = AmpelContext.load("ampel_conf.yaml", tier=2)
AmpelDB.delete_ampel_databases(ctx.config, "AmpelTest")
base_path = "/Users/hu/Documents/ZTF/"

In [3]:
ap = AlertProcessor(
    context = ctx,
    process_name = "test",
    publish_stats = ["mongo"],
    supplier = "ZiAlertSupplier",
    log_profile = "default", # other options: 'default' or 'silent'
    directives = [
        {
            "channel": "VAL_TEST1",
            "stock_update": "ZiStockIngester",
            't0_add': {
                "ingester": "ZiAlertContentIngester",
                "t2_compute": {
                    "ingester": "DualPointT2Ingester",
                    "config": {"tags": ["ZTF"]},
                    "units": [
                        {   # every 3rd photopoint
                            'unit': 'DemoPointT2Unit', 
                            'ingest': {'eligible': {'pps': (2, None, 3)}}
                        }
                    ]
                }
            }
        },
        {
            "channel": "VAL_TEST2",
            "stock_update": "ZiStockIngester",
            't0_add': {
                "ingester": "ZiAlertContentIngester",
                "t2_compute": {
                    "ingester": "DualPointT2Ingester",
                    "config": {"tags": ["ZTF"]},
                    "units": [
                        {
                            # Last photopoint
                            'unit': 'DemoPointT2Unit', 
                            "ingest": {'eligible': {'pps': 'last'}}
                        }
                    ]
                }
            }
        }
    ]
)


2020-06-18 16:51:32 AmpelDB:159 INFO
 Creating AmpelTest_data -> stock
   Creating index: {'index': [('_id', 1), ('channel', 1)], 'args': {'unique': True}}

2020-06-18 16:51:32 AlertProcessor:175 INFO
 AlertProcessor setup completed


In [4]:
ap.iter_max=1
ap.set_loader(TarAlertLoader(f'{base_path}/ztf_public_20200225.tar.gz'))
ap.run()


2020-06-18 16:51:32 AmpelDB:159 INFO
 Creating AmpelTest_ext -> counter

2020-06-18 16:51:32 AmpelDB:159 INFO
 Creating AmpelTest_var -> logs
   Creating index: {'index': [('run', 1)]}
   Creating index: {'index': [('stock', 1)], 'args': {'sparse': True}}
   Creating index: {'index': [('channel', 1)], 'args': {'sparse': True}}

2020-06-18 16:51:32 AmpelDB:159 INFO
 Creating AmpelTest_var -> events

2020-06-18 16:51:32 AmpelDB:159 INFO
 Creating AmpelTest_data -> t2
   Creating index: {'index': [('stock', 1), ('channel', 1)]}
   Creating index: {'index': [('status', 1)]}

2020-06-18 16:51:32 AmpelDB:159 INFO
 Creating AmpelTest_data -> t0
   Creating index: {'index': [('stock', 1)]}

2020-06-18 16:51:33 AmpelDB:159 INFO
 Creating AmpelTest_data -> t1
   Creating index: {'index': [('stock', 1)]}

2020-06-18 16:51:33 CORE AlertProcessor:347 SHOUT [r=1]
 Processing alerts

2020-06-18 16:51:33 CORE IngestionHandler:346 INFO [stock=574072209, channel=['VAL_TEST1', 'VAL_TEST2'], alert=115010

1

In [5]:
print("Datapoints in the T0 collection (1 is oldest):")
dps = sorted(ctx.db.get_collection("t0").find(), key=lambda x: x['body']['jd'], reverse=True)
for i, el in enumerate(dps):
    print(f"{len(dps)-i}: {el['_id']}")

Datapoints in the T0 collection (1 is oldest):
10: 1150106942415010004
9: 1149154782415010008
8: 1149140212415010010
7: 1146225822415010005
6: 1145226652415010008
5: -11451754868220687
4: 1142200132415010009
3: 1139178722415010006
2: 1139141562415010005
1: 1136226592415010011


In [6]:
from ampel.util.pretty import prettyjson, set_bold
t2_docs = list(ctx.db.get_collection("t2").find())
for el in t2_docs: el['_id'] = str(el['_id']) # allow serialization
print("Created T2 documents")
set_bold(prettyjson(t2_docs, indent=3), match=".*link|.*unit")

Created T2 documents


In [7]:
assert t2_docs[0]['link'] == 1139178722415010006
assert t2_docs[1]['link'] == 1146225822415010005
assert t2_docs[2]['link'] == 1150106942415010004
assert "VAL_TEST1" in t2_docs[2]['channel']
assert "VAL_TEST2" in t2_docs[2]['channel']